## Python Text Modeling and TensorFlow Learn

Before you turn this problem in, make sure everything runs as expected. First, restart the kernel (in the menubar, select Kernel → Restart) and then run all cells (in the menubar, select Cell → Run All).  You can speak with others regarding the assignment but all work must be your own. 


### This (plus Separate Spark Part) is a  30 point assignment graded from answers to questions and automated tests that should be run at the bottom. Be sure to clearly label all of your answers and commit final tests at the end. If you attempt to fake passing the tests you will receive a 0 on the assignment and it will be considered an ethical violation. (Note, not all questions have tests).

#### 20 points for this Notebook and 10 points for the Spark notebook. 


### You must show the executed code and then the output . Do not just copy and past the code to a markdown cell. 

In [ ]:
NAME = ""
COLLABORATORS = ["Alyssa Hacker"]  #You can speak with others regarding the assignment, but all typed work must be your own.

In [ ]:
%matplotlib inline
%load_ext ipython_unittest
import matplotlib.pyplot as plt
import numpy as np

### This Data is From Kickstarter
If you don't know Kickstarter, take a few minutes to take a look on the [website](http://www.kickstarter.com).

In [ ]:
#This will fail. Read on to find why.
import pandas as pd
df = pd.read_csv('kickstarter.csv')

1. The below code should work. Explain what the `encoding` term means.  We actually didn't talk much about in class, but it is one of those things that pops up quite frequently when working with text data. OK here is a start. 
[Encoding](https://www.w3.org/International/questions/qa-what-is-encoding)

In [ ]:
#This should work. 
df = pd.read_csv('kickstarter.csv', encoding = 'latin-1')

In [ ]:
#Let's look at the data.
print(df.columns)
df.head()

## The `state` is the variable of interest. This indicates whether a Kickstarter campain has succeeded or failed. 

In [ ]:
#Nex we are going to generate counts by state, which appears to be the most interesting DV. 
df.groupby(['state']).agg('count')

## A couple independent variables are listed below.

In [ ]:
#Nex we are going to generate counts by state, which appears to be the most interesting DV. 
df.groupby(['spotlight']).agg('count')

In [ ]:
#Nex we are going to generate counts by state, which appears to be the most interesting DV. 
<insert text to view each country by count>

In [ ]:
#Nex we are going to generate counts by state, which appears to be the most interesting DV. 
<insert text to view disable_communications by count>

(2). Create `blurb_new` and `name_new` with just the words.  I got you most of the way there but make it **lowercase**. 

(3). Replace the `-` with a space for the slug term and remove any additional spaces (so there is only 1 space between)..

In [ ]:
# OK I'm going  
import re
regex = re.compile('[^a-zA-Z ]')
regex2= re.compile('\s+')
regex3 = re.compile('-')
#First parameter is the replacement, second parameter is your input string
df['blurb_new'] = df['blurb'].apply(lambda x: regex.sub('', str(x)()))
df['blurb_new'] = df['blurb_new'].apply(lambda x: regex2.sub(' ', str(x)()))
#<Fix slug>
#<Fix newname>
#df[['name_new','blurb_new','slug_new']]

(4). Create a count vectorizer called count_slug for the `slug_new` field. Create a dictionary `vocab_count` to include the count for each word in the corpus (example shown below). 

In [ ]:
#<create a count vectorizer>
vocab_count=count_vectorizer.vocabulary_
vocab_count['cybersecurity']


(5). Create a TFIDF vectorizer called `tfidf_vectorizer_blurb` using the `blurb_new` field. Create a dictionary `vocab_tfidf` to include the count for each word in the corpus (example shown below). 

In [ ]:
#<create a tfidf
vocab_tfidf=tfidf_vectorizer.vocabulary_
vocab_tfidf['cybersecurity']


In [ ]:
df.columns

###  Getting Started with Tensorflow Learn
Tensorflow is a little bit senstive as to the data that goes in, which has to be integers. 

In [ ]:
du1=pd.get_dummies(df['spotlight'], drop_first=True)
du2=pd.get_dummies(df['country'], drop_first=True)
X_train = pd.concat([du1, du2], axis=1, join='inner').values

#This is a raw matrix of integers. 
X_train

In [ ]:
#This changes our DV to integers
df['state']=pd.Categorical(df['state'])
y_train=df['state'].cat.codes.real.astype(int)

### Modeling with Tensorflow Learn

In [ ]:
import tensorflow as tf
config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
# List of hidden units per layer. All layers are fully connected. Ex. [64, 32] means first layer has 64 nodes and second one has 32.
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300,100], n_classes=5,
                                         feature_columns=feature_cols, config=config)
dnn_clf = tf.contrib.learn.SKCompat(dnn_clf) # if TensorFlow >= 1.1
dnn_clf.fit(X_train, y_train, batch_size=50, steps=4000)


In [ ]:
from sklearn import metrics
y_pred = dnn_clf.predict(X_train)
#This calculates the accuracy.
accuracy_train=metrics.accuracy_score(y_train,  y_pred['classes'])
print("Accuracy score: ", metrics.accuracy_score(y_train,  y_pred['classes']) )

#0.947420882133

(6) Update the Analysis above to include  'is_starrable' as one of the independent variables

### Challenge! (not for points)

Figure out how to include the text as independent variables through the TFIDF vectorizer.  If you take up the challenge, post the solution to @lack. 

### Tests.
Please run all tests before submitting. 

In [ ]:
%%unittest_main
class TestHomework8(unittest.TestCase):
    def test_text1(self):
        self.assertEqual(df['blurb_new'][2], 'automatically opens your garage door when you come home open close and monitor your garage door from your phone')
    def test_text2(self):
        self.assertEqual(df['slug_new'][2], 'garage beacon ibeacon enabled garage door opener')
    def test_text3(self):
        self.assertEqual(df['name_new'][2], 'garage beacon turn your phone into a garage door remote')
    def test_text4(self):
        self.assertEqual(vocab['cybersecurity'], 1717) 
    def test_text5(self):
        self.assertEqual(vocab_tfidf['cybersecurity'], 2340) 
    def test_text6(self):
        self.assertAlmostEqual(accuracy_train,0.947420882133)    

    